# Initial Watch Records

# 개요

## 1. Web Access

### 1. Modules

In [ ]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크 (https://github.com/ultrafunkamsterdam/undetected-chromedriver)


In [1]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


### 2. Web Access

In [146]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [111]:
# Account Variables

info = open('secure/id_pw.txt').readlines()

account_1 = info[0].split(' ')[1].rstrip('\n')
account_2 = info[2].split(' ')[1].rstrip('\n')


In [147]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
try:
    btn_sign_in = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
    btn_sign_in.click()
except:
    pass

# 계정 이메일 입력
try:
    input_account = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
    input_account.send_keys(account_1)
except:
    pass

# [다음] 버튼 클릭해 패스워드 입력
try:
    btn_next_password = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
    btn_next_password.click()
except:
    pass


In [131]:
# password variables

password_1 = info[1].split(' ')[1].rstrip('\n')
password_2 = info[3].split(' ')[1].rstrip('\n')

In [148]:
# 계정 비밀번호 입력
try:
    input_password = WebDriverWait(driver,20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
    input_password.send_keys(password_1)

except:
    pass

# [다음] 버튼 입력해 로그인 클릭
try:
    btn_next_sign_in = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
    btn_next_sign_in.click()
except:
    pass

del password_1

In [149]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지

try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


In [150]:
driver.refresh()


## 2. Data Collection

### 1. Data Definition

#### 1) Variables

In [42]:
# Variables (1) - 시청기록 태그 속성


# 기본 영상 정보
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>


# 시청비율 판별 정보
nm_class_thumbnail = "OUPWA"  # 기록 우측에 있는 영상 정보 (썸네일) / tag = <div>
nm_class_percentage = "HmLFgd"  # 썸네일 하단 빨간 Percentage Bar / tag = <div>


# 시청기록 - video id
# replace(pattern, '')로 id 값 추출
id_str_pattern = "https://www.youtube.com/watch?v="

'''
WatchDate = container[data-date] / c-wiz 태그의 [data-date] 값
시청비율 확인을 위해 nm_class_RunningTime 활용
'''


'\nWatchDate = container[data-date] / c-wiz 태그의 [data-date] 값\n시청비율 확인을 위해 nm_class_RunningTime 활용\n'

In [43]:
# Variables (2) Youtube Data API


# API 실행에 필요한 모듈
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# API 정보, API Key => API build
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
DEVELOPER_KEY = open('secure/api_key.txt').readline()

youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=DEVELOPER_KEY)


#### 2) Functions

In [44]:
# Function (1) - date

def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d')

get_date('20230203')


datetime.datetime(2023, 2, 3, 0, 0)

In [45]:
# Function (2) - Category Id & Title

""" Google API - Youtube DATA API 활용
    세부 query
    - videos().list(): id 값(watch?v="여기 표시되는 값")으로 동영상 정보 반환 -> 내부의 categoryId 호출
    - videoCategories().list(): categoryId 값(1~44)으로 카테고리 정보 반환 -> 내부의 categoryTitle 호출
"""

""" 개선 필요
        categoryTitle이 영어로 호출됨
        query 옵션에서 한국어를 호출하거나, 번역해야 한다.
"""


def get_catId(input_id):
    video = youtube.videos().list(
        part='snippet',
        id=input_id
    ).execute()

    return video['items'][0]['snippet']['categoryId']


def get_catTitle(num):
    catId = youtube.videoCategories().list(
        id=num,
        part='snippet'
    ).execute()

    try:
        result = catId['items'][0]['snippet']['title']
    except:
        result = ''

    return result


In [46]:
# Function (3) - Watched_YN classifier

""" 구현 가능 Option (1)
    - 구현법
        1. 현재 레코드의 watch_time과 이전 레코드의 watch_time의 차이값 구한다
        2. 영상 길이와 비슷하면 watched, 아닐 시 not_watched

    - 문제점
        1. 마지막으로 시청한 영상은 차이를 구할 수 없다.
        2. 길이 5분 이상 광고를 방치한 경우 시간 계산에 오류가 생긴다
"""

""" 구현 가능 Option (2) -> 채택
    - 구현법
        1. 시청 기록 레코드의 썸네일의 진행도 막대를 활용한다.
        2. 시청 완료하지 않은 영상에 표시되며, 태그 내부의 시청 비율 값을 추출한다.

    - 문제점
        1. 라이브 스트리밍의 경우 진행도 막대가 출력되지 않는다.
        2. 대충 넘기기만 해도 진행도 막대가 표시된다. 
"""


def classify_watched(content):
    running_time = content.find('div', class_=nm_class_RunningTime).text
    thumbnail = content.find('div', class_=nm_class_thumbnail)
    percentage = thumbnail.find('div', class_=nm_class_percentage)

    try:
        result = int(percentage['style'].split(":")[1][:-1]) / 100

    except:
        if len(running_time) > 5:  # 영상 길이가 1시간 이상(0:00:00)인 경우
            result = 0  # 미시청으로 간주
        else:
            result = 1  # 시청으로 간주

    return result


#### 3) Empty DataFrame

In [47]:
import pandas as pd

columns_record = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category_id', 'category_title', 'watched (%)', 'account']
df_record = pd.DataFrame(columns=columns_record)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account


In [48]:
columns_search = ['searchword', 'search_date', 'search_time']
df_search = pd.DataFrame(columns=columns_search)
df_search.head()


,searchword,search_date,search_time


### 2. Collect Web Data

##### (1) 입력 날짜까지의 시청 기록 호출 (03/28 진행 - 미해결)

In [50]:
# 날짜 입력 함수

def get_date_data():
    "4자리 연도, 1~2자리 월/일 숫자를 입력받아 8자리 문자열로 출력. 오입력된 경우 다시 입력하도록 함"
    while True:
        try:
            year, month, day = [int(input(f"{field}를(을) 입력해주세요" )) for field in ("연도(4자리)", "월(1~2자리)", "일(1~2자리)")]
            if not (1000 <= year <= 9999 and 1 <= month <= 12 and 1 <= day <= 31):
                input("Invalid input. Wanna Try again?")
            else:
                break
        except: 
            print("Invalid input. Wanna Try again?")
        
    return f"{year:04d}{month:02d}{day:02d}"

print(f'함수 Test. 입력된 날짜는 {get_date_data()}입니다.')

함수 Test. 입력된 날짜는 20230328입니다.


In [64]:
# 날짜 변환 함수

def convert_date(str_date):
    "'3월 28일'과 같은 형태의 날짜를 8자리 문자열 데이터로 출력"
    nums = [int(each[:-1]) for each in str_date.split(' ')]
    
    if len(nums) == 2: # 연도가 생략된 올해 날짜
        year = datetime.strftime(datetime.today(), '%Y')
        return f"{year}{nums[0]:02d}{nums[1]:02d}"
    
    else:
        return f"{nums[0]}{nums[1]:02d}{nums[2]:02d}"

print(f"Test1. {convert_date('9월 8일')}")
print(f"Test2. {convert_date('2018년 8월 5일')}")


Test1. 20230908
Test2. 20180805


In [72]:
# 현재 호출된 레코드의 마지막 날짜 확인

def get_last_date():
    nm_class_DateDivider = 'MCZgpb' # 날짜 구분 막대

    contents = BeautifulSoup(driver.page_source)
    date_dividers = contents.find_all('div', nm_class_DateDivider)
    
    return int(convert_date(date_dividers[-1].text))

In [119]:
# 입력한 날짜까지의 레코드 호출 - 날짜 구분 막대에 기록된 날짜가 입력된 날짜보다 적을 때 (앞설 때) STOP!

target_date = int(get_date_data())
last_date = int(get_last_date())


while last_date >= target_date:

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//body'))).send_keys(Keys.END)

    last_date = get_last_date()

else:
    pass


In [120]:
target_date, last_date

(20230310, 20230309)

In [ ]:
contents = BeautifulSoup(driver.page_source)

containers = [each for each in contents.find('div', jsname='bN97Pc').children]


**문제**

target_date에 시청 기록이 없으면 어떻게 해야 할까?  
target_date까지의 기록만을 수집하려면 어떻게 해야 할까?


##### (2) 시청 기록 데이터 수집

In [154]:
soup = BeautifulSoup(driver.page_source)
contents = soup.find_all('c-wiz', class_='xDtZAf')
len(contents)

1778

In [155]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = len(df_record)  # df_record의 idx
idx_ser = len(df_search)  # df_search의 idx
# num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime):  # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            channel = content.find('div', class_=nm_class_channel).text
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            watched = classify_watched(content)

            id = content.find('a')['href'].replace(id_str_pattern, '')
            category_id = get_catId(id)
            category_title = get_catTitle(category_id)

            df_record.loc[idx_rec] = [title, channel, running_time, 
                                      watch_date, watch_time, category_id, category_title, watched, account_2]  
            idx_rec += 1

        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Watch Record
1: Watch Record
2: Watch Record
3: Watch Record
4: Watch Record
5: Watch Record
6: Watch Record
7: Watch Record
8: Watch Record
9: Watch Record
10: Watch Record
11: Watch Record
12: Watch Record
13: Watch Record
14: Watch Record
15: Watch Record
16: Watch Record
17: Watch Record
18: Watch Record
19: Watch Record
20: Watch Record
21: Watch Record
22: Watch Record
23: Watch Record
24: Watch Record
25: Watch Record
26: Watch Record
27: Watch Record
28: Watch Record
29: Watch Record
30: Watch Record
31: Watch Record
32: Watch Record
33: Watch Record
34: Watch Record
35: Watch Record
36: Watch Record
37: Watch Record
38: Watch Record
39: Watch Record
40: Watch Record
41: Watch Record
42: Watch Record
43: Watch Record
44: Watch Record
45: Watch Record
46: Watch Record
47: Watch Record
48: Watch Record
49: Watch Record
50: Watch Record
51: Watch Record
52: Watch Record
53: Watch Record
54: Watch Record
55: Watch Record
56: Watch Record
57: Watch Record
58: Watch Record
59: Wat

AttributeError: 'NoneType' object has no attribute 'text'

In [156]:
content

<c-wiz c-wiz="" class="xDtZAf" data-date="20230212" data-hide-ads="true" data-p='%.@.[null,["youtube"]],true]' data-show-delete-individual="true" data-token="AODP23YAAAAaUhgIHRCA-Mjx3Y39Ahj_t6bgn5D9AjoCCADVcN_ATrEaUtaMYbbZOTh5YY5gQw" jsaction="zKRotf:EiRydc;Mipord:N80UCe;" jscontroller="Dlfr9" jsdata="deferred-c1395" jslog="64793; track:impression" jsmodel="hc6Ubd UEAmlc" jsname="Ttx95" jsshadow=""><div aria-label="YouTube 활동 표시 카드" class="GqCJpe u2cbPc LDk2Pd" jsname="MFYZYe" role="listitem" tabindex="-1"><div class="k2bP7e YYajNd"><div class="ztN3hd"><div class="F96K3d"><div class="AYReXd"><img alt="YouTube 로고" class="sPW8R" data-iml="184765.79999995232" height="18" jsaction="error:Lbg96b" jsname="NTz3Fd" src="https://www.gstatic.com/images/branding/product/2x/youtube_24dp.png"/></div><span class="hJ7x8b" jsaction="click:N80UCe" jslog="64799; track:click" jsname="eo0svb">YouTube</span></div><div class="YxbmAc"><div jsaction="JIbuQc:znXRnd;" jscontroller="Fs3gzb"><button aria-label="활

In [157]:
idx_rec, idx_ser, num_ad

(2725, 520, 1142)

In [145]:
driver.close()

In [158]:
df_record.to_csv("data/watch_record_initial.csv", encoding='utf-8')
df_record.tail(3)


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account
2722,DK vs T1 Match22 | 01.30 | 2023 LCK CL 스프링 스플릿,LCK CL,2:54:27,2023-02-12,오후 7:00,20,Gaming,0.1,woghks.study
2723,디플 기아 vs. 한화생명 | 매치39 하이라이트 | 02.12 | 2023 LCK...,LCK,27:09,2023-02-12,오후 6:46,20,Gaming,1.0,woghks.study
2724,나무 그 잡채 | 디플 기아 vs. 한화생명 게임1 하이라이트 | 02.12 | 2...,LCK,8:05,2023-02-12,오후 6:37,20,Gaming,1.0,woghks.study


In [141]:
df_search.to_csv('data/search_record_initial.csv', encoding='utf-8')
df_search.tail(10)


,searchword,search_date,search_time
486,https://www.youtube.com/watch?v=hUWmvqr7PCg,2022-11-18,오후 3:07
487,mbc,2022-11-18,오후 3:07
488,mbx,2022-11-18,오후 3:06
489,대통령실 비서관 mbc,2022-11-18,오후 2:40
490,만 당,2022-11-18,오전 1:03
491,받젭,2022-11-18,오전 12:55
492,받뎁,2022-11-18,오전 12:55
493,팡이요,2022-11-17,오후 11:17
494,https://www.youtube.com/watch?v=UZ-HAfUOZsM,2022-11-17,오후 11:14
495,G식백과,2022-11-17,오후 11:08


In [143]:
print(
    f"입력한 시청기록 레코드는 {num_target_records}개, \n실제 시청한 영상은 {len(df_record)}개, \n광고는 {num_ad}개입니다.")


입력한 시청기록 레코드는 3000개, 
실제 시청한 영상은 1285개, 
광고는 1117개입니다.


In [109]:
driver.close()

### Notes (라고 쓰고 Code recycle bin으로 읽자)

In [ ]:
# 3-3) 입력한 날짜 이전의 레코드 호출

target_date = get_date_data()

btn_calendar = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/div[2]/div[2]/span/div[2]/c-wiz[1]/div/div/div/div/div[1]/button')
btn_calendar.click()

input_date = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input')))
input_date.send_keys(target_date)

btn_apply = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/div[9]/div/div[2]/span/div[2]/div/c-wiz/div/div[4]/div/div/div[2]/div/button/div[3]')))
btn_apply.click()

In [153]:
# 3-3) 입력한 개수만큼 레코드 호출

num_target_records = 3000  
# (오늘 날짜 - 목표 날짜) * 일평균 시청 영상 갯수 * 2.5를 10단위 올림한 수

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
num_current_records = len(soup.find_all('c-wiz', class_='xDtZAf'))


# int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))
import math

num_press_end = int(math.ceil((num_target_records - num_current_records) / 100))

for n in range(num_press_end):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(2)


In [26]:
driver.close()